In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from ipas.create_database.iceagg_collection_bulkinsert import collect_clusters
import numpy as np
from functools import partial
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress
import dask
from scipy import spatial 
import pandas as pd
import pickle
import hickle as hkl
from sqlalchemy import create_engine
import sqlalchemy
from collections import defaultdict  
import os

In [6]:
cluster = SLURMCluster(
queue='batch',
walltime='04-23:00:00',
cores=1,
memory='20000MiB', #1 GiB = 1,024 MiB
processes=1)

cluster.scale(1)
client = Client(cluster)

In [13]:
client

Client Scheduler: tcp://169.226.65.141:42039 Dashboard: http://169.226.65.141:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [10]:
def main():

    delayeds = []
    for r in reqarr:
        for phi in phioarr:
            parallel_clus=partial(collect_clusters, phi, r, nclusters, ncrystals,rand_orient)
            delayeds.append(dask.delayed(collect_clusters)(phi, r, nclusters, ncrystals,rand_orient))
            #collect_clusters(phi, r, nclusters, ncrystals,rand_orient)
    delayeds = client.compute(delayeds)
    output = client.gather(delayeds)
    return output

In [11]:
if __name__ == '__main__':
    phioarr=np.logspace(-2, 2, num=20, dtype=None)#just columns (0,2); plates (-2,0)
    
    reqarr = [1, 2]
    nclusters = 3         #changes how many aggregates per aspect ratio to consider
    ncrystals = 1
    rand_orient = False    #randomly orient the seed crystal and new crystal: uses first random orientation
  
    output = main()


IndexError: list index out of range

In [ ]:
dd = defaultdict(list)
for d in output:
    for key, value in d.items():
        for val in value:
            dd[key].append(val)
    

In [ ]:
%%timeit 
hkl.dump(dd, '../instance_files/createdb_iceagg_flat_r5_9.hkl', mode='w')

In [ ]:
%%timeit
filename = '../instance_files/createdb_iceagg_flat_r5_9'
filehandler = open(filename, 'wb')
pickle.dump(dd, filehandler)
filehandler.close()

In [ ]:
# Dump to file
hkl.dump(dd, '../instance_files/createdb_iceagg_rand_r80_400.hkl', mode='w')

# Dump data, with compression
hkl.dump(dd, '../instance_files/createdb_iceagg_rand_r80_400_gzip.hkl', mode='w', compression='gzip')

# Compare filesizes
print('uncompressed: %i bytes' % os.path.getsize('../instance_files/createdb_iceagg_rand_r80_400.hkl'))
print('compressed:   %i bytes' % os.path.getsize('../instance_files/createdb_iceagg_rand_r80_400_gzip.hkl'))


In [ ]:
with open('../instance_files/instance_iceice_flat_rall', "ab") as f:
    for data in agg_as, agg_bs, agg_cs, phi2Ds, cplxs, dds:
        pickle.dump(data, f)

In [ ]:
aggregates = db[ "aggregates" ]
result=aggregates.insert_many(split)

In [ ]:
%%time 
engine = create_engine('sqlite:///../db_files_round2/createdb_iceagg_rand_r1_4.sqlite')#fast_executemany=True
df = pd.DataFrame.from_dict(dd)
df.to_sql('aggregates', con=engine, if_exists='replace', method='multi',\
          dtype={"ncrytals": sqlalchemy.types.INTEGER(),
                      "points": sqlalchemy.types.PickleType,
                      "a": sqlalchemy.types.Float(precision=6, asdecimal=True),
                      "b": sqlalchemy.types.Float(precision=6, asdecimal=True),
                      "c": sqlalchemy.types.Float(precision=6, asdecimal=True), 
                      "cplx": sqlalchemy.types.Float(precision=6, asdecimal=True),
                      "phi2D": sqlalchemy.types.Float(precision=6, asdecimal=True),
                      "mono_phi": sqlalchemy.types.Float(precision=6, asdecimal=True),
                      "mono_r": sqlalchemy.types.INTEGER()})